## In this ETL Project a messy sales data set and item reference sheet will be used to extract transform and load data into a database management system. 

### The sales data set includes a customer id, sale year, month, fiscal year, fiscal quarter, item id, product group, number of units sold, sales in dollars, and a very messy description field. A item id reference table will also be used and cleaned, and customer data will be extracted from the original sales data file to create a more organized database. 

### Three cleaned tables will be loaded into PostgreSQL 

In [1]:
import pandas as pd

## 1) Clean messy sales data set

In [2]:
# Path data sets
sales_path = "messysalesdata.csv"

# Read csv in original format
messy_sales = pd.read_csv(sales_path)
messy_sales.head()

,Sell-to Customer No_,yr,mo,FiscalYear,FiscalQuarter,No_,Posting Group,units,sales,CustomerType,Description,Your Item Type Field!
0,3XL001,2014,1,2014,Q4,800501-0208,FG SEED KI,300,"$2,100.00",Retail,"SK, 7-Pod, Cherry Tomato",0.0
1,3XL001,2014,1,2014,Q4,800528-0208,FG SEED KI,100,$700.00,Retail,"SK, 7 Pod, Grow Anything",0.0
2,3XL001,2014,1,2014,Q4,800544-0208,FG SEED KI,300,"$2,100.00",Retail,"SK, 7- Pod, Chili Pepper",0.0
3,3XL001,2014,1,2014,Q4,970133-0100,ACCESSORY,30,$465.00,Retail,"Acc, AeroVoir with Stand",0.0
4,3XL001,2014,5,2015,Q1,800500-0208,FG SEED KI,500,"$3,500.00",Retail,"SK, 7- Pod, Gourmet Herb",0.0


In [3]:
# Begin cleaning data set
# Drop Your Item Type Field! Column which contains no data and FiscalYear which is redundant
clean_sales = messy_sales.drop(columns=["Your Item Type Field!", "FiscalYear"])

In [4]:
# Clean column names - uniform formatting and clear titles
clean_sales.columns = clean_sales.columns.str.strip().str.lower()\
    .str.replace(' ', '').str.replace('(', '').str.replace(')', '')\
    .str.replace('-', '').str.replace('_', '')

clean_sales.rename(columns = {'no':'itemno', 'selltocustomerno':'customerno'}, inplace = True) 

print(clean_sales.columns)

Index(['customerno', 'yr', 'mo', 'fiscalquarter', 'itemno', 'postinggroup',
       'units', 'sales', 'customertype', 'description'],
      dtype='object')


In [5]:
# Data type conversions
clean_sales.dtypes

# Clean units and sales to remove objects in str that prevent conversion to numeric
clean_sales["units"] = clean_sales['units'].str.strip().str.replace(' ', '')\
    .str.replace(',', '').str.replace('.', '').str.replace('$', "")\
    .str.replace('-', '').str.replace('(', '').str.replace(')', '')

clean_sales["sales"] = clean_sales['sales'].str.strip().str.replace(' ', '')\
    .str.replace(',', '').str.replace('.', '').str.replace('$', "")\
    .str.replace('-', '').str.replace('(', '').str.replace(')', '')

# Change to correct data types - possible now without symbols in string
clean_sales["units"] = pd.to_numeric(clean_sales["units"])
clean_sales["sales"] =pd.to_numeric(clean_sales["sales"])

print(clean_sales.dtypes)

customerno        object
yr                 int64
mo                 int64
fiscalquarter     object
itemno            object
postinggroup      object
units            float64
sales            float64
customertype      object
description       object
dtype: object


In [6]:
# selltocustomerno column cleaning exploration
print(clean_sales["customerno"].unique())

['3XL001' 'ACE001' 'AGOPS' 'AGSALES' 'AM3001' 'AMA001' 'AMA002' 'AMA003'
 'AMA004' 'AMA005' 'AMA006' 'AMA007' 'AMA008' 'AMA009' 'AMA010' 'BED001'
 'BED003' 'BED004' 'BED005' 'CAN002' 'CIN001' 'COS001' 'CHE004' 'COS003'
 'COS005' 'CRA001' 'DIR105' 'COS002' 'COS004' 'DIR107' 'BJS001' 'DIR108'
 'DIR109' 'DIR111' 'FRY001' 'GMA001' 'GRO004' 'HAW002' 'HAY001' 'HOM011'
 'HOM014' 'HAM001' 'HSN001' 'HUD001' 'HOM015' 'HYD010' 'KOH001' 'KOH002'
 'LOW001' 'LOW002' 'LNL001' 'MAC001' 'MAC002' 'MEH001' 'MEH002' 'MIL003'
 'PRO004' 'QVC001' 'QVC003' 'MCG001' 'MEI001' 'SAM001' 'SAM002' 'SCO003'
 'SHA002' 'SIR001' 'SOC001' 'SPR003' 'STE003' 'SCO005' 'SCO006' 'SUR001'
 'SUR002' 'TAR001' 'TAR002' 'TAS001' 'TRA002' 'TRU001' 'WAL002' 'WAL005'
 'WAL006' 'WAL007' 'WAY002' 'WIL002' 'WOO002' 'WOO003' 'ZUL001']


In [7]:
# fiscalquarter column cleaning exploration
print(clean_sales["fiscalquarter"].unique())

['Q4' 'Q1' 'Q2' 'Q3']


In [8]:
# no. cleaning
print(clean_sales["itemno"].unique())

['800501-0208' '800528-0208' '800544-0208' ... '903121-1100' '900819-1100'
 '900822-1100']


In [9]:
# postinggroup cleaning
clean_sales["postinggroup"] = clean_sales['postinggroup'].str.strip().str.replace(' ', '').str.lower()

print(clean_sales["postinggroup"].unique())

# Replace ag3garden entry with fggardens - ag3garden is a model and belongs under fggardens designation
clean_sales["postinggroup"] = clean_sales["postinggroup"].str.replace('ag3garden', 'fggardens')
print(clean_sales["postinggroup"].unique())

['fgseedki' 'accessory' 'fggardens' 'raw' 'displays' 'seedpod' 'component'
 'ag3garden']
['fgseedki' 'accessory' 'fggardens' 'raw' 'displays' 'seedpod' 'component']


In [10]:
# customertype cleaning
clean_sales["customertype"] = clean_sales["customertype"].str.lower()
print(clean_sales["customertype"].unique())


['retail' 'dr']


In [11]:
# Begin to clean a very messy description column
clean_sales["description"] = clean_sales['description'].str.strip()\
    .str.replace('.', '').str.replace('$', '')\
    .str.replace('(', '').str.replace(')', '').str.lower()\
    .str.replace('-', '').str.replace('pod', 'p')\
    .str.replace('sk', '').str.replace('acc', '').str.replace('rp', '')\
    .str.replace('pack', 'pk').str.replace('w/', '').str.replace('us', '')\
    .str.replace('eu', '').str.replace('china', '').str.replace('dlx', '')\
    .str.replace('seedstartingsystem', 'sss').str.replace('seedstartingsys', 'sss')\
    .str.replace('aggrowbowlv2growmedia', 'growbowlandgrowmedia').str.replace('ag', '')\
    .str.replace('display', '').str.replace('seedp', '').str.replace('std', '')\
    .str.replace('pop', '').str.replace('aerogdn', '').str.replace('aerogarden', '')\
    .str.replace('&', '').str.replace('/', '').str.replace("''", "").str.replace('pdq', '')\
    .str.replace('versal', '').str.replace('bat', '').str.replace('deluxe', '').str.replace('wb', '')\
    .str.replace('gherb', 'gh').str.replace('gourmetherb', 'gh').str.replace(' ','' )
# Observe more description cleaning needs
clean_sales.head()

,customerno,yr,mo,fiscalquarter,itemno,postinggroup,units,sales,customertype,description
0,3XL001,2014,1,Q4,800501-0208,fgseedki,300.0,210000.0,retail,",7p,cherrytomato"
1,3XL001,2014,1,Q4,800528-0208,fgseedki,100.0,70000.0,retail,",7p,growanything"
2,3XL001,2014,1,Q4,800544-0208,fgseedki,300.0,210000.0,retail,",7p,chilipepper"
3,3XL001,2014,1,Q4,970133-0100,accessory,30.0,46500.0,retail,",aerovoirwithstand"
4,3XL001,2014,5,Q1,800500-0208,fgseedki,500.0,350000.0,retail,",7p,gourmetherb"


In [12]:
# More order dependent cleaning - replace all commas with spaces, then strip() to remove spaces from bookends of entry
clean_sales["description"] = clean_sales["description"].str.replace(',', ' ').str.strip()

# Replace spaces inside string to be able to separate cleaner values within description column
clean_sales["description"] = clean_sales["description"].str.replace(' ', ',')
clean_sales.head()

,customerno,yr,mo,fiscalquarter,itemno,postinggroup,units,sales,customertype,description
0,3XL001,2014,1,Q4,800501-0208,fgseedki,300.0,210000.0,retail,"7p,cherrytomato"
1,3XL001,2014,1,Q4,800528-0208,fgseedki,100.0,70000.0,retail,"7p,growanything"
2,3XL001,2014,1,Q4,800544-0208,fgseedki,300.0,210000.0,retail,"7p,chilipepper"
3,3XL001,2014,1,Q4,970133-0100,accessory,30.0,46500.0,retail,aerovoirwithstand
4,3XL001,2014,5,Q1,800500-0208,fgseedki,500.0,350000.0,retail,"7p,gourmetherb"


In [13]:
separated_sales = clean_sales["description"].str.split(",", expand=True)
separated_df = pd.DataFrame(separated_sales)
separated_df.head()

separated_df.to_csv("separated_description.csv")

## 2) Clean itemno reference data

In [14]:
item_path = "itemlookup.csv"
messy_item = pd.read_csv(item_path)
messy_item.head()

,ItemNumber,Posting Group,Description,Add Your Item Type Field Here!
0,0020-00Z,FG SEED KI,"SK, 14 UnivPods, Salad Lover's",NaN
1,0021-00Z,FG SEED KI,"SK, 14P, Herb Lover's Seed Kit",NaN
2,100235-0000,RAW,Flat Plant Spacer (China),NaN
3,100824-0000,RAW,SuperGrow Nutrients (3oz.),NaN
4,100828-0000,ACCESSORY,Aerovoir Stand,NaN


In [15]:
# Drop unneccessary columns
clean_item = messy_item.drop(columns=["Add Your Item Type Field Here!"])

# Clean column names - uniform formatting and clear titles
clean_item.columns = clean_item.columns.str.strip().str.lower()\
    .str.replace(' ', '').str.replace('(', '').str.replace(')', '')\
    .str.replace('-', '').str.replace('_', '')

clean_item.rename(columns = {'itemnumber':'itemno'}, inplace = True) 

# Make sure clean_sales columns match clean_item keys
print(clean_item.columns)
print(clean_sales.columns)

Index(['itemno', 'postinggroup', 'description'], dtype='object')
Index(['customerno', 'yr', 'mo', 'fiscalquarter', 'itemno', 'postinggroup',
       'units', 'sales', 'customertype', 'description'],
      dtype='object')


In [16]:
# check that dtypes are correct
clean_item.dtypes

itemno          object
postinggroup    object
description     object
dtype: object

In [17]:
# itemno cleaning exploration
print(clean_item['itemno'].unique())

# strip just incase
clean_item['itemno'] = clean_item['itemno'].str.strip()


['0020-00Z' '0021-00Z' '100235-0000' ... '976926-0100' '976927-0000'
 '976950-0100']


In [18]:
# postinggroup cleaning
clean_item['postinggroup'].unique()
clean_item["postinggroup"] = clean_item['postinggroup'].str.strip().str.replace(' ', '').str.lower()

# Make sure clean_sales columns match clean_item postinggroup keys
print(clean_item["postinggroup"].unique())
print(clean_sales["postinggroup"].unique())

# Item reference data has more postinggroup options than sales data - item reference will/ 
# contain posting group data for clarity and most likely accuracy - real world situation would require/
# a conversation with database administrator

['fgseedki' 'raw' 'accessory' 'fggardens' 'seedpod' 'displays' 'seedkits'
 'aerogarden' 'component']
['fgseedki' 'accessory' 'fggardens' 'raw' 'displays' 'seedpod' 'component']


In [19]:
# description cleaning clean_item with same methods as clean_sales
clean_item["description"] = clean_item['description'].str.strip()\
    .str.replace('.', '').str.replace('$', '')\
    .str.replace('(', '').str.replace(')', '').str.lower()\
    .str.replace('-', '').str.replace('pod', 'p')\
    .str.replace('sk', '').str.replace('acc', '').str.replace('rp', '')\
    .str.replace('pack', 'pk').str.replace('w/', '').str.replace('us', '')\
    .str.replace('eu', '').str.replace('china', '').str.replace('dlx', '')\
    .str.replace('seedstartingsystem', 'sss').str.replace('seedstartingsys', 'sss')\
    .str.replace('aggrowbowlv2growmedia', 'growbowlandgrowmedia').str.replace('ag', '')\
    .str.replace('display', '').str.replace('seedp', '').str.replace('std', '')\
    .str.replace('pop', '').str.replace('aerogdn', '').str.replace('aerogarden', '')\
    .str.replace('&', '').str.replace('/', '').str.replace("''", "").str.replace('pdq', '')\
    .str.replace('versal', '').str.replace('bat', '').str.replace('deluxe', '').str.replace('wb', '')\
    .str.replace('gherb', 'gh').str.replace('gourmetherb', 'gh').str.replace(' ', '')
# Observe more description cleaning needs
clean_item.head()

,itemno,postinggroup,description
0,0020-00Z,fgseedki,",14univps,saladlover's"
1,0021-00Z,fgseedki,",14p,herblover'sseedkit"
2,100235-0000,raw,flatplantspacer
3,100824-0000,raw,supergrownutrients3oz
4,100828-0000,accessory,aerovoirstand


In [20]:
# More order dependent cleaning - replace all commas with spaces, then strip() to remove spaces from bookends of entry
clean_item["description"] = clean_item["description"].str.replace(',', ' ').str.strip().str.replace('  ', ' ')

clean_item.head()

,itemno,postinggroup,description
0,0020-00Z,fgseedki,14univps saladlover's
1,0021-00Z,fgseedki,14p herblover'sseedkit
2,100235-0000,raw,flatplantspacer
3,100824-0000,raw,supergrownutrients3oz
4,100828-0000,accessory,aerovoirstand


In [21]:
# Separate description into columns to make data more useful
separated_item = clean_item["description"].str.split(" ", expand=True)
separated_item.head()

# Find which columns have uneccessary data
print(separated_item[0].unique())
print("................................................................")
print("................................................................")

print(separated_item[1].unique())
print("................................................................")
print("................................................................")

print(separated_item[2].unique())
print("................................................................")
print("................................................................")

print(separated_item[3].unique())
print("................................................................")
print("................................................................")

print(separated_item[4].unique())
print("................................................................")
print("................................................................")

separated_item.drop(columns=[4])


['14univps' '14p' 'flatplantspacer' 'supergrownutrients3oz'
 'aerovoirstand' '7aerator100610' 'seedp' '7p' '7' '7ghcherrytomato'
 '7led' '3' '3sl' 'sproutplblk' 'ctc' "macy's" 'bbb' "kohl's" 'costco'
 "macy's2019" 'kohls' 'bedhbeyond' 'acehardware' 'ctc12pallet'
 'ctc14pallet' 'herbie' 'harvestelite' 'harvestplgrey'
 '3slwalmartdsplyprod' '6' 'plant' 'extra' 'extraled' '3p'
 '7pmegacherrytomato' 'seedstartingsystem' 'seedstartingsys'
 'seedstartingsystemsprout' 'seedstartingsystemharvest'
 'seedstartingsystemharvst' 'seedstartingsystemharvstfr'
 'seedstartingsystembounty' '2x3p' '2x' '3pviolayellowredwing'
 '3pvibrantviolas_2pk' '3pzinniamix' 'uni' 'growanything25pk'
 'growanything50pk' '6p' 'seedstartingsystem6' 'gardenstartertray'
 'seedstartingsystem6plstc' '7pgourmetherbseedkit' '12p' 'gf'
 '6pgrowanything' '6phrlmsaladgreens' '9p' '9pmightymini'
 '9phrlmsaladgreens' 'ultraledbundle' 'farm' '3penguin' '7silver' '3psl'
 '3pslwhite' 'sprout' 'sprt' 'sproutred' 'sproutled' 'farmpl' 'f

,0,1,2,3
0,14univps,saladlover's,None,None
1,14p,herblover'sseedkit,None,None
2,flatplantspacer,None,None,None
3,supergrownutrients3oz,None,None,None
4,aerovoirstand,None,None,None
5,7aerator100610,blk,None,None
6,seedp,7p,celosia,None
7,seedp,tomato,None,None
8,seedp,holybasil,None,None
9,seedp,basil,genov,None


In [22]:
# Add in separated descriptions to clean_item
clean_item["description1"] = separated_item[0]
clean_item["description2"] = separated_item[1]
clean_item["description3"] = separated_item[2]
clean_item["description4"] = separated_item[3]

In [23]:
# export clean_item to csv
clean_item.to_csv("cleanitemlookup.csv", index=False)

## 3) Clean customerno reference data

In [24]:
customer_path = "customerlookup.csv"
messy_customer = pd.read_csv(customer_path)
messy_customer.head()

,Sell-to Customer No_,CustomerType
0,3XL001,Retail
1,ACE001,Retail
2,AGOPS,Retail
3,AGSALES,Retail
4,AM3001,Retail


In [25]:
# Clean columns
clean_customer = messy_customer

clean_customer.columns = clean_customer.columns.str.strip().str.lower()\
    .str.replace(' ', '').str.replace('(', '').str.replace(')', '')\
    .str.replace('-', '').str.replace('_', '')

clean_customer.rename(columns = {'selltocustomerno':'customerno'}, inplace = True) 

# Keys match
print(clean_customer.columns)
print(clean_item.columns)
print(clean_sales.columns)

Index(['customerno', 'customertype'], dtype='object')
Index(['itemno', 'postinggroup', 'description', 'description1', 'description2',
       'description3', 'description4'],
      dtype='object')
Index(['customerno', 'yr', 'mo', 'fiscalquarter', 'itemno', 'postinggroup',
       'units', 'sales', 'customertype', 'description'],
      dtype='object')


In [26]:
# Clean customerno
print(clean_customer["customerno"].unique())

clean_customer["customerno"] = clean_customer["customerno"].str.strip()

['3XL001' 'ACE001' 'AGOPS' 'AGSALES' 'AM3001' 'AMA001' 'AMA002' 'AMA003'
 'AMA004' 'AMA005' 'AMA006' 'AMA007' 'AMA008' 'AMA009' 'AMA010' 'BED001'
 'BED003' 'BED004' 'BED005' 'BJS001' 'CAN002' 'CHE004' 'CIN001' 'COS001'
 'COS002' 'COS003' 'COS004' 'COS005' 'CRA001' 'DIR011' 'DIR105' 'DIR107'
 'DIR108' 'DIR109' 'DIR110' 'DIR111' 'FRY001' 'GLO002' 'GMA001' 'GRO004'
 'HAM001' 'HAW002' 'HAY001' 'HOM011' 'HOM014' 'HOM015' 'HSN001' 'HUD001'
 'HYD005' 'HYD010' 'HYD011' 'KOH001' 'KOH002' 'LIF002' 'LNL001' 'LOW001'
 'LOW002' 'MAC001' 'MAC002' 'MCG001' 'MEH001' 'MEH002' 'MEI001' 'MIL003'
 'OPE001' 'PRO004' 'QVC001' 'QVC003' 'SAM001' 'SAM002' 'SCO003' 'SCO005'
 'SCO006' 'SER002' 'SHA002' 'SIR001' 'SOC001' 'SPR003' 'STE003' 'SUR001'
 'SUR002' 'TAR001' 'TAR002' 'TAS001' 'TRA002' 'TRU001' 'WAL002' 'WAL005'
 'WAL006' 'WAL007' 'WAY002' 'WIL002' 'WOO002' 'WOO003' 'ZUL001']


In [27]:
# Clean customertype
print(clean_customer["customertype"].unique())

clean_customer["customertype"] = clean_customer["customertype"].str.strip().str.lower()

print(clean_customer["customertype"].unique())

['Retail' 'DR']
['retail' 'dr']


In [28]:
# export clean_customer to csv
clean_customer.to_csv("cleancustomerlookup.csv", index=False)

## 4) Create paired down sales data table for reference



In [29]:
print(clean_sales.columns)
# remove 'postinggroup', 'description', and 'customertype' which can be accessed through customerreference and item reference

Index(['customerno', 'yr', 'mo', 'fiscalquarter', 'itemno', 'postinggroup',
       'units', 'sales', 'customertype', 'description'],
      dtype='object')


In [30]:
clean_sales = clean_sales[['itemno', 'customerno', 'yr', 'mo', 'fiscalquarter', 'units', 'sales' ]]
clean_sales.head()

,itemno,customerno,yr,mo,fiscalquarter,units,sales
0,800501-0208,3XL001,2014,1,Q4,300.0,210000.0
1,800528-0208,3XL001,2014,1,Q4,100.0,70000.0
2,800544-0208,3XL001,2014,1,Q4,300.0,210000.0
3,970133-0100,3XL001,2014,1,Q4,30.0,46500.0
4,800500-0208,3XL001,2014,5,Q1,500.0,350000.0


In [31]:
# Export clean_sales dataframe to csv
clean_sales.to_csv('cleansalesdata.csv', index=False) 